In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D
import matplotlib.pyplot as plt


In [ ]:
#insert path to HAM10000 folder
ham10000Path = ''
processed = os.path.join(ham10000Path, 'processed')

In [ ]:
seed = 69420
trainingDS = image_dataset_from_directory(
    processed,
    image_size = (600, 450), 
    validation_split = 0.1,
    subset = 'training',
    seed = seed,
    label_mode = 'categorical'
)

testingDS = image_dataset_from_directory(
    processed, 
    image_size = (600, 450), 
    validation_split = 0.1,
    subset = 'validation',
    seed = seed,
    label_mode = 'categorical'
)

In [ ]:
model = Sequential()
model.add(Conv2D(16,3,padding="same", activation="relu", input_shape=(600, 450, 3)))
model.add(MaxPooling2D(pool_size = 2, strides = 2))

model.add(Conv2D(16,3,padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size = 2, strides = 2))

model.add(Conv2D(16,3,padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size = 2, strides = 2))

model.add(Conv2D(16,3,padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size = 2, strides = 2))

model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(7, activation = "softmax"))

loss = 'categorical_crossentropy'
model.compile(loss = loss, optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001), metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
baseCounts = np.array([327, 514, 1099, 115, 1113, 6705, 142])

inverses = 1./baseCounts

newWeights = inverses * 10015

indices = [0, 1, 2, 3, 4, 5, 6]

weights = dict(zip(indices, newWeights))
print(weights)

In [ ]:
filepath = 'my_best_model.epoch{epoch:02d}-loss{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(
    filepath=filepath, 
    monitor='val_loss',
    verbose=1, 
    save_best_only=True,
    mode='min'
)

callbacks = [checkpoint]

In [ ]:
epoch = 100

data = model.fit(
    trainingDS,
    epochs = epoch,
    validation_data = testingDS,
    class_weight = weights,
    callbacks = callbacks
)

In [ ]:
trainAccuracy = data.history['accuracy']
trainLoss = data.history['loss']
testAccuracy = data.history['val_accuracy']
testLoss = data.history['val_loss']

In [ ]:
epochRange = range(epoch)
plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochRange, trainAccuracy, label='Training Accuracy')
plt.plot(epochRange, testAccuracy, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochRange, trainLoss, label='Training Loss')
plt.plot(epochRange, testLoss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [19]:
y_true = []
y_pred = []
for x,y in testingDS:
  y= tf.argmax(y,axis=1)
  y_true.append(y)
  y_pred.append(tf.argmax(model.predict(x),axis = 1))
  
y_pred = tf.concat(y_pred, axis=0)
y_true = tf.concat(y_true, axis=0)

matrix =  tf.math.confusion_matrix(labels = y_true, predictions = y_pred)
print(matrix)

tf.Tensor(
[[  7   8  12   0   3   2   0]
 [  5  17   7   0   6   5   1]
 [  7  20  62   3  15  22   1]
 [  1   3   4   2   0   0   0]
 [  2   6  25   0  32  49   0]
 [  5  18  66   1  43 525   0]
 [  0   3   0   0   2   2   9]], shape=(7, 7), dtype=int32)
